# Lesson 2: Tool Calling

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [1]:
from dotenv import load_dotenv

load_dotenv()

import os

os.environ["OPENAI_API_KEY"] = "my_api_key"

## 1. Define a Simple Tool

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Add two numbers together"""
    return x + y

def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)


In [5]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="llama3.2")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function when x = 2 and y = 3",
    verbose=True,
)

print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 3}
=== Function Output ===
25
25


## 2. Define an Auto-Retrieval Tool

In [11]:
Settings.llm = Ollama(model="llama3.2")

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-03-08
last_modified_date: 2025-03-08

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4, Jinlin Wang1, Zili Wang, Steven Ka Shing Yau5, Zijuan Lin4,
Liyang Zhou6, Chenyu Ran1, Lingfeng Xiao1,7, Chenglin Wu1†, J¨urgen Schmidhuber2,8
1DeepWisdom, 2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University, 4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University, 6University of Pennsylvania,
7University of California, Berkeley, 8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks

In [9]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)

query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [13]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)

In [14]:
print(str(response))

MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with high accuracy. Specifically, it performs exceptionally well in Pass@1 evaluations, demonstrating its robustness and efficiency in handling complex software projects. Additionally, MetaGPT showcases its ability to complete tasks at a 100% rate, highlighting its superior performance compared to other popular frameworks.


In [15]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-03-08', 'last_modified_date': '2025-03-08'}


### Define the Auto-Retrieval Tool

In [16]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [17]:
llm = Ollama(model="llama3.2")

response = llm.predict_and_call(
    [vector_query_tool],
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True,
)
print(str(response))

=== Calling Function ===
Calling function: vector_tool with args: {"page_numbers": "BLANK", "query": "MetaGPT page 2 summary"}
=== Function Output ===
Empty Response
Empty Response


In [18]:
for n in response.source_nodes:
    print(n.metadata)

In [19]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    )
)

In [20]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_numbers": "['BLANK', '8']", "query": "MetaGPT comparisons with ChatDev"}
=== Function Output ===
MetaGPT outperforms ChatDev in nearly all metrics on the challenging SoftwareDev dataset, achieving higher scores in terms of executability and productivity. Additionally, it requires less time and has a lower cost of human revision compared to ChatDev. MetaGPT's ability to generate code efficiently is also notable, needing significantly fewer tokens than ChatDev for each line of code.


In [21]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-03-08', 'last_modified_date': '2025-03-08'}


In [22]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "paper"}
=== Function Output ===
Encountered error: 
